In [38]:
import pandas as pd
import os,sys, imp
sys.path.append('./src/')
import doc_utils_clean as doc_utils
import feature_utils
import defines

In [5]:
doc_db =  pd.read_csv("./dataframes/doc_db.csv")
doc_db.sort_values(by='doc_idx_from_name',inplace=True)
doc_db

,path,file_name,doc_idx_from_name
3,./tmp/01_aingimel3_lc.docx,01_aingimel3_lc.docx,1
2,./tmp/02_aingimel12_lc.docx,02_aingimel12_lc.docx,2
1,./tmp/03_aingimel26_lc.docx,03_aingimel26_lc.docx,3
7,./tmp/04_nun2_lc.docx,04_nun2_lc.docx,4
5,./tmp/05_kafhey_23_lc.docx,05_kafhey_23_lc.docx,5
6,./tmp/06_lamedbet21_lc.docx,06_lamedbet21_lc.docx,6
0,./tmp/07_alefsameh3_lc.docx,07_alefsameh3_lc.docx,7
4,./tmp/08_kafdalet_15_lc.docx,08_kafdalet_15_lc.docx,8


In [18]:
for i in range(1,9):
    feature_utils.get_and_save_sent_lemma_db(i)

In [76]:
imp.reload(feature_utils)
feature_utils.tfidf_build()

TfIdf 4 saved
TfIdf 2 saved
TfIdf 5 saved
TfIdf 3 saved
TfIdf 8 saved
TfIdf 6 saved
TfIdf 7 saved
TfIdf 1 saved


### Reshape list to sequences of 6-10... sentences

In [184]:
imp.reload(feature_utils)
feature_utils.load_doc_features(2)
X_doc,y_doc,groups_doc = feature_utils.pack_doc_sentences(2)
X_02,y_02,groups_02 = feature_utils.reshape_doc_features_to_sequence(X_doc,y_doc,groups_doc,6,6)

2 doc 186 sentences packed
2 doc sentences reshaped: from 186 to 31


In [176]:
X_try,y_try,groups_try = feature_utils.pack_doc_features(2)

In [187]:
len(X_02[0])

6

In [223]:
X_02[0]

[{'is_client': 0,
  'sent_len': 33.0,
  'TOKEN': 6,
  'POSTAG_AT': 0,
  'POSTAG_BN': 1,
  'POSTAG_BNT': 0,
  'POSTAG_CC': 0,
  'POSTAG_CD': 0,
  'POSTAG_CDT': 0,
  'POSTAG_CONJ': 0,
  'POSTAG_COP': 0,
  'POSTAG_DEF': 1,
  'POSTAG_DT': 0,
  'POSTAG_DTT': 1,
  'POSTAG_EX': 0,
  'POSTAG_IN': 1,
  'POSTAG_INTJ': 0,
  'POSTAG_JJ': 0,
  'POSTAG_JJT': 0,
  'POSTAG_MD': 0,
  'POSTAG_NCD': 0,
  'POSTAG_NN': 2,
  'POSTAG_NNP': 1,
  'POSTAG_NNT': 0,
  'POSTAG_POS': 0,
  'POSTAG_PREPOSITION': 0,
  'POSTAG_PRP': 0,
  'POSTAG_QW': 0,
  'POSTAG_RB': 0,
  'POSTAG_REL': 0,
  'POSTAG_S_PRN': 0,
  'POSTAG_TEMP': 0,
  'POSTAG_VB': 0,
  'f_gen_F': 2,
  'f_gen_M': 2,
  'f_num_D': 0,
  'f_num_P': 0,
  'f_num_S': 4,
  'f_suf_gen_F': 0,
  'f_suf_gen_M': 0,
  'f_suf_num_P': 0,
  'f_suf_num_S': 0,
  'f_suf_per_1.0': 0,
  'f_suf_per_2.0': 0,
  'f_suf_per_3.0': 0,
  'f_per_1': 0,
  'f_per_2': 0,
  'f_per_3': 0,
  'f_per_A': 1,
  'f_tense_BEINONI': 0,
  'f_tense_FUTURE': 0,
  'f_tense_IMPERATIVE': 0,
  'f_tense_PAS

In [188]:
len(X_try[0])

6

In [195]:
X_6_6,y_6_6,groups_6_6 = feature_utils.pack_all_doc_sentences(6,6)
len(X_6_6),len(y_6_6),len(groups_6_6)

7 doc 448 sentences packed
7 doc sentences reshaped: from 448 to 75
3 doc 152 sentences packed
3 doc sentences reshaped: from 152 to 26
2 doc 186 sentences packed
2 doc sentences reshaped: from 186 to 31
1 doc 618 sentences packed
1 doc sentences reshaped: from 618 to 103
8 doc 259 sentences packed
8 doc sentences reshaped: from 259 to 44
5 doc 375 sentences packed
5 doc sentences reshaped: from 375 to 63
6 doc 471 sentences packed
6 doc sentences reshaped: from 471 to 79
4 doc 543 sentences packed
4 doc sentences reshaped: from 543 to 91
Sentenced packed for 8 docs


(512, 512, 512)

In [196]:
y_6_6[0]

['False', 'False', 'False', 'False', 'False', 'False']

In [218]:
from operator import itemgetter
def get_prediction(train_idx,test_idx,X,y):
    crf = CRF(
    min_freq = 5,
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    )
    print ("Test idx: {}..., train idx: {}...".format(train_idx[:10],test_idx[:10]))
    crf.fit(itemgetter(*train_idx)(X), itemgetter(*train_idx)(y))
    y_pred  =  crf.predict(itemgetter(*test_idx)(X))
    labels = list(crf.classes_)
    f1 = metrics.flat_f1_score(itemgetter(*test_idx)(y), y_pred,average='weighted', labels=labels)
    recall = metrics.flat_recall_score(itemgetter(*test_idx)(y), y_pred,average='weighted', labels=labels)
    precision = metrics.flat_precision_score(itemgetter(*test_idx)(y), y_pred,average='weighted', labels=labels)
    return (f1,recall,precision)

In [217]:
def leaveOneOut_validate(X,y,groups,seq_len,step):
    logo = LeaveOneGroupOut()
    score = [get_prediction(train_idx,test_idx,X,y)  for train_idx, test_idx in logo.split(X, y, groups)]
    return score

In [214]:
from sklearn_crfsuite import scorers, CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.model_selection import LeaveOneGroupOut

for train_idx, test_idx in logo.split(X_6_6, y_6_6, groups_6_6):
    print ("len train {}, len test {}".format(len(train_idx),len(test_idx)))


len train 409, len test 103
len train 481, len test 31
len train 486, len test 26
len train 421, len test 91
len train 449, len test 63
len train 433, len test 79
len train 437, len test 75
len train 468, len test 44


In [211]:
score_db = pd.DataFrame()


In [216]:
def add_score_to_db(prefix,score):
    score_db.loc[prefix,"f1"] = np.mean(score[:][0])
    score_db.loc[prefix,"recall"] = np.mean(score[:][1])
    score_db.loc[prefix,"precision"] = np.mean(score[:][2])

In [221]:
imp.reload(feature_utils)
shapes = [(6,6),(6,3),(6,1)]
X,y,group =  feature_utils.pack_all_doc_sentences()
for shape in shapes:
    seq_len = shape[0]
    step = shape[1]
    X_shaped,y_shaped,groups_shaped = feature_utils.reshape_doc_features_to_sequence(X,y,groups,seq_len,step)
    score = leaveOneOut_validate(X_shaped,y_shaped,groups_shaped,seq_len,step)
    add_score_to_db("{}_{}".format(seq_len,step),score)

7 doc 448 sentences packed
3 doc 152 sentences packed
2 doc 186 sentences packed
1 doc 618 sentences packed
8 doc 259 sentences packed
5 doc 375 sentences packed
6 doc 471 sentences packed
4 doc 543 sentences packed
3052 sentenced packed for 8 docs
7 doc sentences reshaped: from 3052 to 509
Test idx: [0 1 2 3 4 5 6 7 8 9]..., train idx: [131 132 133 134 135 136 137 138 139 140]...
Test idx: [0 1 2 3 4 5 6 7 8 9]..., train idx: [100 101 102 103 104 105 106 107 108 109]...
Test idx: [0 1 2 3 4 5 6 7 8 9]..., train idx: [75 76 77 78 79 80 81 82 83 84]...
Test idx: [0 1 2 3 4 5 6 7 8 9]..., train idx: [419 420 421 422 423 424 425 426 427 428]...
Test idx: [0 1 2 3 4 5 6 7 8 9]..., train idx: [278 279 280 281 282 283 284 285 286 287]...
Test idx: [0 1 2 3 4 5 6 7 8 9]..., train idx: [340 341 342 343 344 345 346 347 348 349]...
Test idx: [75 76 77 78 79 80 81 82 83 84]..., train idx: [0 1 2 3 4 5 6 7 8 9]...
Test idx: [0 1 2 3 4 5 6 7 8 9]..., train idx: [234 235 236 237 238 239 240 241 242 

In [222]:
score_db

,f1,recall,precision
6_6,0.696805,0.837201,0.876303
6_3,0.709099,0.803827,0.920085
6_1,0.717101,0.837365,0.909433
